In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torchvision import datasets

import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Đang sử dụng thiết bị: {device}")

Đang sử dụng thiết bị: cuda


In [16]:
train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_df = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [17]:
print(train_df.shape)

(42000, 785)


In [18]:
targets = train_df['label']
features = train_df.drop('label', axis = 1) / 255.0

In [19]:
targets_tensor = torch.tensor(targets.values).long()
features_tensor = torch.tensor(features.values).float()

test_features = test_df / 255.0
test_tensor = torch.tensor(test_features.values).float()
test_tensor = test_tensor.view(-1, 1, 28, 28)

print(features_tensor.shape)
print(targets_tensor.shape)

torch.Size([42000, 784])
torch.Size([42000])


In [20]:
class MNISTDataset(Dataset):
    def __init__(self, features, targets, transform=None):
        self.features = features
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        # 1. Lấy dòng dữ liệu thứ 'index'
        pixel_array = self.features[index]
        
        # 2. Biến đổi hình dạng (Reshape)
        image = pixel_array.view(1, 28, 28) 
        
        # 3. Nếu có hàm biến đổi (transform) thì áp dụng nó
        if self.transform:
            image = self.transform(image)
            
        # 4. Trả về ảnh và nhãn
        label = self.targets[index]
        return image, label

train_transform = transforms.Compose([
    transforms.RandomRotation(degrees=10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))
])

In [21]:
train_dataset = MNISTDataset(features_tensor, targets_tensor, transform = train_transform)
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True, num_workers=2, pin_memory=True)

In [22]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3)
        self.bn1 = nn.BatchNorm2d(32)
        
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size = 2)

        
        self.fc1 = nn.Linear(in_features= 1152, out_features= 128)
        self.fc2 = nn.Linear(in_features= 128, out_features= 10)
        self.dropout = nn.Dropout(p=0.5)
        # --- PHẦN 1: KHAI BÁO CÁC LỚP ---
        # Ví dụ: self.conv1 = nn.Conv2d(...)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.pool(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = torch.relu(x)
        
        x = x.view(-1, 3 * 3 * 128)

        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        # --- PHẦN 2: SẮP XẾP LUỒNG ĐI ---
        # Ví dụ: x = self.conv1(x)
        return x

In [23]:
model = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [24]:
num_epochs = 30
model.train()
1
for epoch in range(num_epochs):
    epoch_loss = 0
    for images, labels in train_loader:
        images = images.to(device) 
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    
    scheduler.step()
    avg_loss = epoch_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

Epoch [1/30], Loss: 0.2949
Epoch [2/30], Loss: 0.1180
Epoch [3/30], Loss: 0.0954
Epoch [4/30], Loss: 0.0810
Epoch [5/30], Loss: 0.0801
Epoch [6/30], Loss: 0.0716
Epoch [7/30], Loss: 0.0671
Epoch [8/30], Loss: 0.0624
Epoch [9/30], Loss: 0.0601
Epoch [10/30], Loss: 0.0588
Epoch [11/30], Loss: 0.0418
Epoch [12/30], Loss: 0.0374
Epoch [13/30], Loss: 0.0333
Epoch [14/30], Loss: 0.0327
Epoch [15/30], Loss: 0.0315
Epoch [16/30], Loss: 0.0301
Epoch [17/30], Loss: 0.0299
Epoch [18/30], Loss: 0.0285
Epoch [19/30], Loss: 0.0297
Epoch [20/30], Loss: 0.0293
Epoch [21/30], Loss: 0.0278
Epoch [22/30], Loss: 0.0271
Epoch [23/30], Loss: 0.0253
Epoch [24/30], Loss: 0.0264
Epoch [25/30], Loss: 0.0249
Epoch [26/30], Loss: 0.0247
Epoch [27/30], Loss: 0.0249
Epoch [28/30], Loss: 0.0246
Epoch [29/30], Loss: 0.0240
Epoch [30/30], Loss: 0.0236


In [27]:
model.eval()
predictions = []

test_tensor = test_tensor.to(device)

with torch.no_grad():
    outputs = model(test_tensor)
    _, predicted = torch.max(outputs, 1)
    predictions = predicted.cpu().numpy()

# Tạo file submission
submission = pd.DataFrame({
    'ImageId': range(1, len(predictions) + 1),
    'Label': predictions
})

# Lưu ra file CSV
submission.to_csv('submission.csv', index=False)
print("Đã tạo file submission.csv thành công! Bạn có thể tải về và nộp lên Kaggle.")

Đã tạo file submission.csv thành công! Bạn có thể tải về và nộp lên Kaggle.


In [26]:
# --- LƯU MÔ HÌNH ĐÃ TRAIN VÀO FILE .pth ---

MODEL_SAVE_PATH = 'model.pth'

# Lưu lại chỉ state_dict (các tham số đã học)
torch.save(model.state_dict(), MODEL_SAVE_PATH) 

print(f"Đã lưu các tham số của mô hình vào file: {MODEL_SAVE_PATH}")

Đã lưu các tham số của mô hình vào file: model.pth
